In [ ]:
!pip install accelerate bitsandbytes transformers

In [ ]:
pip install -U transformers

In [ ]:
# pip install bitsandbytes accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

access_token = "" # 9b

quantization_config = BitsAndBytesConfig(load_in_8bit=True)

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-9b", token=access_token)
model = AutoModelForCausalLM.from_pretrained("google/gemma-2-9b", quantization_config=quantization_config, token=access_token)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/39.1k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/4.84G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

...

model-00006-of-00008.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00007-of-00008.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/2.38G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

We will save the big boy and then run these codes on separate notebooks

In [ ]:
import json
nar_data = json.load(open("/content/best_big_boy.json"))

In [ ]:
narrations = []

for vid in nar_data["videos"]:
    for clip in vid["clips"]:
        for ann in clip["annotations"]:
            for query in ann["language_queries"]:
                narrations.append(query.get("query", None))

print(narrations[0:5])
print(len(narrations))

['#C C removes the brush on the paint box', '#C C paints a drawing book with the brush', '#C C mixes the paint on the paint box with the brush', '#C C touches the paint on the paint box with the brush', '#C C removes the brush on the paint box']
22038


In [ ]:
def iterate_in_batches(lst, batch_size=16):
    for i in range(0, len(lst), batch_size):
        yield lst[i:i + batch_size]

# Example usage:
my_list = [i for i in range(100)]  # Sample list
for batch in iterate_in_batches(my_list, 32):
    print(len(batch))

32
32
32
4


In [ ]:
import random
random.seed(42)

In [ ]:
random_integers = [random.randint(0, len(narrations)) for _ in range(30)]

random_narrations = [narrations[i] for i in random_integers]

In [ ]:
random_narrations

['#O The man A holds the belt with his left hand.',
 '#C C puts the pen in the bag ',
 '#C C drops corn in bin',
 '#C C picks the garlic from the bag',
 '#C C walks around ',
 '#C C holds  a bucket',
 '#C C places the pillow case on the bed.',
 '#C C applies oil',
 '#C C picks the utensils from the cabinet',
 '#C C pours the mixture on the pan',
 '#C C paints the wood ornament with the paint brush in her right hand.',
 '#C C  stares at the room.',
 '#C C picks the whisky ',
 '#C C scoops some soup in the pot with the spoon. #unsure',
 '#C C drops the small ceramic plate in the plate rack',
 '#C C looks around ',
 '#C C knits the cloth with the pair of knitting needles and the yarn in both hands.',
 '#C C places her right hand on the nylon.',
 '#C C lifts the hand',
 '#C C moves a food item on the napkin.',
 '#C C sweeps the floor with the broom in her right hand',
 '#C C  paints on the wood',
 '#O man b drops the mug on the table',
 '#C C passes the brick in his eft hand to his right h

# MY trial

In [ ]:
questions = []
# we need better model
for batch in iterate_in_batches(random_narrations, 10):
  text=""
  for i in range(len(batch)):
    text+= f"{i+1}. " + batch[i] + "\n"


  input_text = f"""Given the following narrations describing the actions of a person, generate independent from one another {len(batch)}
simple queries (one per line per narration) that could be answered by looking at the narrations.
Use where, what, when, and who questions only. The questions must follow one of the templates provided below.
Do not ask for feedback. You MUST only generate {len(batch)} questions, strictly following these templates:

TEMPLATES:
objects - Where is object X before / after event Y?
objects - Where is object X?
objects - Where did I put in X?
objects - How many X's?
objects - What X did I Y?
objects - In what location did I see object X?
objects - What X is Y?
objects - state of an object
objects - Where is my object X?
Place - Where did I put X?
People - who did I interact with when I did activity X?
People - Who did I talk to in location X?
People - When did I interact with person with role X?

******************************************
EXAMPLE:
Narrations:
1. #C C irons cloth on a table
2. #C C interacts with X

Corresponding Questions:
1. What does C iron?
2. Who does C interact with?
******************************************

Now it is your turn
Narrations:
{text}

Corresponding Questions:
"""
  input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

  # generated_text=""
  # while generated_text.count("?") < 2 + len(batch):
  # outputs = model.generate(**input_ids, max_length=500, num_return_sequences=1, do_sample=False, top_k=50, top_p=0.95, temperature=0.7)
  outputs = model.generate(**input_ids, max_length=500)
  generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

  print(generated_text)
  print("--------------------------------------------------------------------------------------")

  trimmed_text = generated_text[generated_text.rfind("1."):]
  trimmed_text = trimmed_text.strip()
  indexed_questions = trimmed_text.split("\n")
  for q in indexed_questions:
    questions.append(q[3:])

# print("\n\n\n ------------------------- Question Answer Pairs ----------------------------------")
# for q, a in zip(questions, random_narrations):
#   print(q, "->", a)


Given the following narrations describing the actions of a person, generate independent from one another 10 
simple queries (one per line per narration) that could be answered by looking at the narrations.
Use where, what, when, and who questions only. The questions must follow one of the templates provided below. 
Do not ask for feedback. You MUST only generate 10 questions, strictly following these templates:

TEMPLATES:
objects - Where is object X before / after event Y?
objects - Where is object X?
objects - Where did I put in X?
objects - How many X's?
objects - What X did I Y?
objects - In what location did I see object X?
objects - What X is Y?
objects - state of an object
objects - Where is my object X?
Place - Where did I put X?
People - who did I interact with when I did activity X?
People - Who did I talk to in location X?
People - When did I interact with person with role X?

******************************************
EXAMPLE:
Narrations:
1. #C C irons cloth on a table
2. #

In [ ]:
questions = []
# we need a better model
for batch in iterate_in_batches(random_narrations, 10):
    text = ""
    for i in range(len(batch)):
        text += f"{i+1}. " + batch[i] + "\n"

    input_text = f"""Given the following narrations describing the actions of a person, generate independent from one another {len(batch)}
simple queries (one per line per narration) that could be answered by looking at the narrations.
Use where, what, when, and who questions only. The questions must follow one of the templates provided below.
Do not ask for feedback. You MUST only generate {len(batch)} questions, strictly following these templates:

TEMPLATES:
objects - Where is object X before / after event Y?
objects - Where is object X?
objects - Where did I put in X?
objects - How many X's?
objects - What X did I Y?
objects - In what location did I see object X?
objects - What X is Y?
objects - state of an object
objects - Where is my object X?
Place - Where did I put X?
People - who did I interact with when I did activity X?
People - Who did I talk to in location X?
People - When did I interact with person with role X?

Now it is your turn
Narrations:
{text}

Corresponding Questions:
"""
    input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")
    outputs = model.generate(**input_ids, max_length=500)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    print(generated_text)
    print("--------------------------------------------------------------------------------------")

    trimmed_text = generated_text[generated_text.rfind("1."):]
    trimmed_text = trimmed_text.strip()
    indexed_questions = trimmed_text.split("\n")
    for q in indexed_questions:
        questions.append(q[3:])

# print("\n\n\n ------------------------- Question Answer Pairs ----------------------------------")
# for q, a in zip(questions, random_narrations):
#   print(q, "->", a)


Given the following narrations describing the actions of a person, generate independent from one another 10 
simple queries (one per line per narration) that could be answered by looking at the narrations.
Use where, what, when, and who questions only. The questions must follow one of the templates provided below. 
Do not ask for feedback. You MUST only generate 10 questions, strictly following these templates:

TEMPLATES:
objects - Where is object X before / after event Y?
objects - Where is object X?
objects - Where did I put in X?
objects - How many X's?
objects - What X did I Y?
objects - In what location did I see object X?
objects - What X is Y?
objects - state of an object
objects - Where is my object X?
Place - Where did I put X?
People - who did I interact with when I did activity X?
People - Who did I talk to in location X?
People - When did I interact with person with role X?

Now it is your turn
Narrations:
1. #O The man A holds the belt with his left hand.
2. #C C puts the

# Save below for history

In [ ]:
questions = []
# we need better model
for batch in iterate_in_batches(random_narrations, 10):
  text=""
  for i in range(len(batch)):
    text+= f"{i+1}. " + batch[i] + "\n"


  input_text = f"""Given the following narrations describing the actions of a person, generate independent from one another {len(batch)}
  simple queries (one per line per narration) that could be answered by looking at the narrations.
  Use where, what , when and who questions only. Don't ask for feedback, you MUST only generate {len(batch)} questions.

******************************************
EXAMPLE:
Narrations:
1. #C C irons cloth on a table
2. #C C interacts with X

Corresponding Questions:
1. What does C iron?
2. Who does c interact with?
******************************************

Now it is your turn
Narrations:
{text}

Corresponding Questions:
"""
  input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

  # generated_text=""
  # while generated_text.count("?") < 2 + len(batch):
  # outputs = model.generate(**input_ids, max_length=500, num_return_sequences=1, do_sample=False, top_k=50, top_p=0.95, temperature=0.7)
  outputs = model.generate(**input_ids, max_length=500)
  generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

  print(generated_text)
  print("--------------------------------------------------------------------------------------")

  trimmed_text = generated_text[generated_text.rfind("1."):]
  trimmed_text = trimmed_text.strip()
  indexed_questions = trimmed_text.split("\n")
  for q in indexed_questions:
    questions.append(q[3:])

# print("\n\n\n ------------------------- Question Answer Pairs ----------------------------------")
# for q, a in zip(questions, random_narrations):
#   print(q, "->", a)


Given the following narrations describing the actions of a person, generate independent from one another 10 simple queries (one per line per narration) that could be answered by looking at the narrations.
Use where, what , when and who questions only. Don't ask for feedback, you MUST only generate 10 questions.

******************************************
EXAMPLE:
Narrations:
1. #C C irons cloth on a table
2. #C C interacts with X

Corresponding Questions:
1. What does C iron?
2. Who does c interact with?
******************************************

Now it is your turn
Narrations:
1. #O The man A holds the belt with his left hand.
2. #C C puts the pen in the bag 
3. #C C drops corn in bin
4. #C C picks the garlic from the bag
5. #C C walks around 
6. #C C holds  a bucket
7. #C C places the pillow case on the bed.
8. #C C applies oil
9. #C C picks the utensils from the cabinet
10. #C C pours the mixture on the pan


Corresponding Questions:
1. What is the person holding the belt?
2. Who i

In [ ]:
questions = []
# we need better model
for batch in iterate_in_batches(random_narrations, 10):
  text=""
  for i in range(len(batch)):
    text+= f"{i+1}. " + batch[i] + "\n"


  input_text = f"""Given the following narrations describing the actions of a person, generate independent from one another {len(batch)} simple queries (one per line per narration) that could be answered by looking at the narrations.
Use where, what , when and who questions only. Don't ask for feedback, you MUST only generate {len(batch)} questions.

******************************************
EXAMPLE:
Narrations:
1. #C C irons cloth on a table
2. #C C interacts with X

Corresponding Questions:
1. What does C iron?
2. Who does c interact with?
******************************************

Now it is your turn
Narrations:
1. the person waters the plants using a water gun
2. the person touches the plant with the left hand
3. the person waters the plants using a water gun
4. the person stares at the plant
5. the person touches the soil with the left hand


Corresponding Questions:
"""
  input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

  # generated_text=""
  # while generated_text.count("?") < 2 + len(batch):
  # outputs = model.generate(**input_ids, max_length=500, num_return_sequences=1, do_sample=False, top_k=50, top_p=0.95, temperature=0.7)

  outputs = model.generate(**input_ids, max_length=500)
  generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

  print(generated_text)
  print("--------------------------------------------------------------------------------------")

  trimmed_text = generated_text[generated_text.rfind("1."):]
  trimmed_text = trimmed_text.strip()
  indexed_questions = trimmed_text.split("\n")
  for q in indexed_questions:
    questions.append(q[3:])

# print("\n\n\n ------------------------- Question Answer Pairs ----------------------------------")
# for q, a in zip(questions, random_narrations):
#   print(q, "->", a)


Given the following narrations describing the actions of a person, generate independent from one another 10 simple queries (one per line per narration) that could be answered by looking at the narrations.
Use where, what , when and who questions only. Don't ask for feedback, you MUST only generate 10 questions.

******************************************
EXAMPLE:
Narrations:
1. #C C irons cloth on a table
2. #C C interacts with X

Corresponding Questions:
1. What does C iron?
2. Who does c interact with?
******************************************

Now it is your turn
Narrations:
1. the person waters the plants using a water gun
2. the person touches the plant with the left hand
3. the person waters the plants using a water gun
4. the person stares at the plant
5. the person touches the soil with the left hand


Corresponding Questions:
1. What does the person do with a water gun?
2. Which hand does the person touch the plant with?
3. What is the person doing when watering the plants?
4

In [ ]:
questions = []

input_text = f"""Prompt:
Given the following narrations describing the actions of a person, Generate a set of simple queries (one per line) that could be answered by looking at the video segments corresponding to these narrations:
1. the person waters the plants using a water gun
2. the person touches the plant with the left hand
3. the person waters the plants using a water gun
4. the person stares at the plant
5. the person touches the soil with the left hand
Generated text:
"""
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

# generated_text=""
# while generated_text.count("?") < 2 + len(batch):
# outputs = model.generate(**input_ids, max_length=500, num_return_sequences=1, do_sample=False, top_k=50, top_p=0.95, temperature=0.7)

outputs = model.generate(**input_ids, max_length=500)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(generated_text)
print("--------------------------------------------------------------------------------------")

# trimmed_text = generated_text[generated_text.rfind("1."):]
# trimmed_text = trimmed_text.strip()
# indexed_questions = trimmed_text.split("\n")
# for q in indexed_questions:
#   questions.append(q[3:])

# print("\n\n\n ------------------------- Question Answer Pairs ----------------------------------")
# for q, a in zip(questions, random_narrations):
#   print(q, "->", a)


Prompt: 
Given the following narrations describing the actions of a person, Generate a set of simple queries (one per line) that could be answered by looking at the video segments corresponding to these narrations:
1. the person waters the plants using a water gun
2. the person touches the plant with the left hand
3. the person waters the plants using a water gun
4. the person stares at the plant
5. the person touches the soil with the left hand
Generated text:
1. What is the person doing with the water gun?
2. Which hand is touching the plant?
3. What is the person doing with the water gun?
4. What is the person looking at?
5. Which hand is touching the soil?
--------------------------------------------------------------------------------------


# My trial

In [ ]:
# def generate_questions_from_answers(text_narrations_list):
#   #print(text_narrations_list)
#   questions = []
#   text=""
#   # we need better model
#   '''
#   example
#    1. C irons cloth on a table
#    2. C interacts with X
#    '''
#   for i in range(len(text_narrations_list)):
#     text+= f"{i+1}. " + text_narrations_list[i] + "\n"


#   input_text = f"""Given the following narrations describing the actions of a person, generate independent from one another {len(text_narrations_list)}
#   simple queries (one per line per narration) that could be answered by looking at the narrations.
#   Use where, what , when and who questions only. Don't ask for feedback, you MUST only generate {len(text_narrations_list)} questions.

#   ******************************************
#   EXAMPLE:
#   Narrations:
#   1. #C C irons cloth on a table
#   2. #C C interacts with X

#   Corresponding Questions:
#   1. What does C iron?
#   2. Who does C interact with?
#   ******************************************

#   Now it is your turn
#   Narrations:
#   {text}

#   Corresponding Questions:
#   """
#   input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

#   outputs = model.generate(**input_ids, max_length=1000, num_return_sequences=1, do_sample=False, top_k=50, top_p=0.95, temperature=0.7)
#   generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
#   #print(generated_text)
#   trimmed_text = generated_text[generated_text.rfind(":")+1:]
#   trimmed_text = trimmed_text.strip()
#   indexed_questions = trimmed_text.split("\n")
#   for q in indexed_questions:
#     questions.append(q.strip()[3:])

#   #print(questions)
#   #print(text_narrations_list)
#   return questions

In [ ]:
# # we are going to work with a copy
# !cp big_boy.json big_boy_copy.json

In [ ]:
# import json

# big_boy = json.load(open("/content/big_boy_copy.json"))
# big_boy_vids = big_boy["videos"]

# slice_size = 128

# # Initialize the list to hold the slices
# slices_list = []

# # Loop through the original list and create slices
# for i in range(0, len(big_boy_vids), slice_size):
#     slice_part = big_boy_vids[i:i + slice_size]
#     slices_list.append(slice_part)

In [ ]:
# print("Insert an integer between 0 and", len(slices_list)-1)

Insert an integer between 0 and 16


In [ ]:
# SPLIT_INDEX = 15 # change here !!!
# used_vids = slices_list[SPLIT_INDEX]

In [ ]:
# for vid in used_vids:
#   for clip in vid["clips"]:
#     for annotator in clip["annotations"]:
#       language_queries = annotator["language_queries"]
#       answers = [item["query"] for item in language_queries]
#       questions = generate_questions_from_answers(answers)

#       while len(questions) < len(language_queries):
#         questions.append("") # empty but at least no bug!!!

#       for i in range(len(language_queries)):
#         language_queries[i]["query"] = questions[i]

In [ ]:
# #Save the part
# new_file_name ="NLQ_pretrain_part_"+str(SPLIT_INDEX) + ".json"

# with open(new_file_name, 'w') as file:
#     json.dump(used_vids, file, indent=4)